In [9]:
# ------------------------------------------------------------------------------------------------------------------------ #
# ---------------------------------------- 0.import library ---------------------------------------- # 
# ------------------------------------------------------------------------------------------------------------------------ #

# module : 설치가 되어 있지 않으면 설치 필요
import sys
import os
import xlrd
import pandas as pd
import numpy as np
import statistics
import math
import datetime
import time
import re # regural expression
import functools
import random
import gc
import pickle
import warnings
import scipy
import sympy as sym

#
from sys import stdout
from functools import reduce # apply function simultaneously on all elements  
from io import StringIO
from pandas.api.types import CategoricalDtype # convert categorical data type 
from itertools import groupby
from itertools import product
from itertools import chain
from itertools import cycle, islice
from collections import defaultdict
from datetime import datetime # set datetime class 
from collections import Counter
from scipy import stats 

### plot ###
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns 
from matplotlib.ticker import MultipleLocator, IndexLocator, FuncFormatter
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib import gridspec

# 
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import r2_score
from scipy.signal import savgol_filter
from scipy.ndimage.filters import gaussian_filter1d # line smoothing    
from tqdm import tqdm
import xgboost


# plot에 한글 사용을 위해서 세팅 : plot에서 한글이 깨지는 경우 font를 직접 지정 해 주어야 함 
from matplotlib import font_manager, rc

# 
font_path = "C:/Windows/Fonts/malgun.ttf" # 맑은고딕
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font)


# check module version
print("- python version :", sys.version)# 파이썬 버전
print("- pandas version :", pd.__version__) # pandas 버전
print("- seaborn version :", sns.__version__) # seaborn 버전
print("- matplotlib version :", matplotlib.__version__) # matplotlib 버전


# set options
pd.set_option('display.max_row', 100) # row 100개까지 display 
pd.set_option('display.max_columns', 120) # column 120개까지 display

# set work directory
# os.chdir(r"C:\Users\2019002\Desktop\전기로국책과제")

# 
print("- dir path :", os.getcwd())

# ignore warnings
warnings.filterwarnings("ignore")
# ------------------------------------------------------------------------------------------------------------------------ #

- python version : 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
- pandas version : 1.4.4
- seaborn version : 0.11.2
- matplotlib version : 3.5.2
- dir path : C:\Users\2019010\Desktop\AI전기로_국책\PCS\3차년도


C:\Users\2019010\AppData\Local\Temp\11\ipykernel_59724\3847409579.py:51: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d # line smoothing


In [10]:
# percentile for aggregate summary 

def trim_mean(n):
    def trim_mean_(x):
        return stats.trim_mean(x, n)
    ;
    
    trim_mean_.__name__ = 'trim_mean_%s' % n
    return trim_mean_
;


def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    ;
    
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_
;

# calculate time delta 
def fn_tmp_timesec(data) :

    # sorting 
    data = data.sort_values("Time", ascending = True)

    # 
    total_prc_tm = (max(data.Time) - min(data.Time)).total_seconds()
    data["Timedelta"] = [max(data.Time) - x for x in data["Time"]]
    data["Timedsec"] = [total_prc_tm - x.total_seconds() for x in data["Timedelta"]]
    data["Time_minute"] = [x/60 for x in data["Timedsec"]]
    
    retrun_dt = data[df_cols + ["Timedelta", "Timedsec", "Time_minute"]] 
    return retrun_dt
;


''

## 전압구간 집계 데이터 불러오기 


In [19]:
# 
# os.chdir(r"D:\국책과제_2차년도\DAT\전압인가순서적용\arc요약")1
os.chdir(r"\\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\2차년도\OUT\csv\전압인가순서적용\arc요약")



# -- 2022
# agg_arc_2022 = pd.read_csv(r"2022_HEAT별 arc요약_전압인가순서적용.csv", header = 0, encoding = "cp949")

# -- 2023 
agg_arc_2023 = pd.read_csv(r"2023_HEAT별 arc요약_전압인가순서적용.csv", header = 0, encoding = "cp949")
# agg_arc_2023 = pd.read_csv(r"2023_01_06_HEAT별 arc요약_전압인가순서적용.csv", header = 0, encoding = "cp949")


In [20]:
# agg_arc_2022.shape, 
agg_arc_2023.shape

(120049, 394)

In [21]:
# agg_arc_2022["Year"] = 2022
agg_arc_2023["Year"] = 2023

In [6]:
# 
agg_arc_alldata = pd.concat([agg_arc_2022, agg_arc_2023], axis = 0)

# 2022년 3월 데이터 제거
agg_arc_alldata = agg_arc_alldata[~agg_arc_alldata["HEAT_NO"].isin(agg_arc_2022[agg_arc_2022["Month"] == 3]["HEAT_NO"].unique())]

In [22]:
agg_arc_alldata = agg_arc_2023
agg_arc_alldata["Year"] = 2023

In [7]:
# 데이터 확인 
agg_arc_alldata[agg_arc_alldata["Month"] == 10]

,Month,HEAT_NO,ActualCHRG_PX3,ActFceTap_Px3,전압번호순서_Tag,전압인가순서,Current1_min,Current1_median,Current1_max,Current1_mean,Current1_std,Current2_min,Current2_median,Current2_max,Current2_mean,Current2_std,Current3_min,Current3_median,Current3_max,Current3_mean,Current3_std,Voltage1_min,Voltage1_median,Voltage1_max,Voltage1_mean,Voltage1_std,Voltage2_min,Voltage2_median,Voltage2_max,Voltage2_mean,Voltage2_std,Voltage3_min,Voltage3_median,Voltage3_max,Voltage3_mean,Voltage3_std,ZFeedback1_min,ZFeedback1_median,ZFeedback1_max,ZFeedback1_mean,ZFeedback1_std,ZFeedback2_min,ZFeedback2_median,ZFeedback2_max,ZFeedback2_mean,ZFeedback2_std,ZFeedback3_min,ZFeedback3_median,ZFeedback3_max,ZFeedback3_mean,ZFeedback3_std,Res1_min,Res1_median,Res1_max,Res1_mean,Res1_std,Res2_min,Res2_median,Res2_max,Res2_mean,...,B1_O2ShroudFlow_min,B1_O2ShroudFlow_median,B1_O2ShroudFlow_max,B1_O2ShroudFlow_mean,B1_O2ShroudFlow_std,B2_O2ShroudFlow_min,B2_O2ShroudFlow_median,B2_O2ShroudFlow_max,B2_O2ShroudFlow_mean,B2_O2ShroudFlow_std,B4_O2ShroudFlow_min,B4_O2ShroudFlow_median,B4_O2ShroudFlow_max,B4_O2ShroudFlow_mean,B4_O2ShroudFlow_std,SFCE_CO2_Pct_min,SFCE_CO2_Pct_median,SFCE_CO2_Pct_max,SFCE_CO2_Pct_mean,SFCE_CO2_Pct_std,SFCE_CO_Pct_min,SFCE_CO_Pct_median,SFCE_CO_Pct_max,SFCE_CO_Pct_mean,SFCE_CO_Pct_std,SFCE_H2O_Pct_min,SFCE_H2O_Pct_median,SFCE_H2O_Pct_max,SFCE_H2O_Pct_mean,SFCE_H2O_Pct_std,SFCE_Offgas_MassFlow_min,SFCE_Offgas_MassFlow_median,SFCE_Offgas_MassFlow_max,SFCE_Offgas_MassFlow_mean,SFCE_Offgas_MassFlow_std,NormH2OPctFlow_min,NormH2OPctFlow_median,NormH2OPctFlow_max,NormH2OPctFlow_mean,NormH2OPctFlow_std,NormCO2PctFlow_min,NormCO2PctFlow_median,NormCO2PctFlow_max,NormCO2PctFlow_mean,NormCO2PctFlow_std,NormCOPctFlow_min,NormCOPctFlow_median,NormCOPctFlow_max,NormCOPctFlow_mean,NormCOPctFlow_std,COCO2Ratio_Clus_min,COCO2Ratio_Clus_median,COCO2Ratio_Clus_max,COCO2Ratio_Clus_mean,COCO2Ratio_Clus_std,PCFactor_min,PCFactor_median,PCFactor_max,PCFactor_mean,PCFactor_std
65263,10,E233892,1,7,7_V1,1.0,18.27,47.468750,87.06,47.829340,14.482387,2.58,49.312500,82.00,49.448788,16.676891,31.22,50.875000,69.75,50.923500,10.380248,258.5,410.000,602.5,403.76947,83.586336,229.4,367.500,614.5,377.23053,79.237365,173.2,367.5000,467.2,349.34903,69.708696,3.07,8.421875,32.75,10.085988,6.100699,3.10,8.031250,100.00,10.790381,14.007495,3.27,6.839844,13.92,7.187779,2.255340,1.099609,7.011719,25.703125,7.942193,4.617951,2.070312,6.269531,100.000000,8.275695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65264,10,E233892,1,8,8_V1,2.0,21.12,56.875000,65.00,49.807293,16.993397,35.25,60.796875,76.10,56.583332,16.938326,34.72,49.703125,58.28,49.302082,8.325311,272.2,342.125,642.5,406.04166,148.386608,308.5,361.125,445.8,367.79166,45.952534,193.1,409.7500,492.0,386.85416,103.550725,4.19,5.785156,25.40,10.602214,8.694505,4.43,6.205078,12.10,7.243490,3.108561,3.31,8.796875,11.28,8.195638,2.874639,2.640625,4.724609,21.203125,8.759114,7.578890,3.189453,5.056641,9.562500,5.896484,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65265,10,E233892,1,9,9_V1,3.0,9.57,33.875000,65.00,40.007812,23.167348,33.03,43.562500,79.25,52.937500,21.682467,37.40,43.531250,50.25,43.568750,5.726745,267.0,515.000,645.0,443.85000,167.923122,295.8,415.750,489.8,394.15000,73.959998,321.5,397.2500,578.0,405.05000,103.151042,4.11,15.398438,67.40,21.675000,26.248352,3.73,8.031250,14.83,8.804688,4.498587,7.39,8.226562,15.05,9.482813,3.153543,2.269531,13.632812,47.281250,15.960938,18.355607,2.460938,6.460938,10.781250,6.813281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [16]:
agg_arc_2023

,Month,HEAT_NO,ActualCHRG_PX3,ActFceTap_Px3,전압번호순서_Tag,전압인가순서,Current1_min,Current1_median,Current1_max,Current1_mean,Current1_std,Current2_min,Current2_median,Current2_max,Current2_mean,Current2_std,Current3_min,Current3_median,Current3_max,Current3_mean,Current3_std,Voltage1_min,Voltage1_median,Voltage1_max,Voltage1_mean,Voltage1_std,Voltage2_min,Voltage2_median,Voltage2_max,Voltage2_mean,Voltage2_std,Voltage3_min,Voltage3_median,Voltage3_max,Voltage3_mean,Voltage3_std,ZFeedback1_min,ZFeedback1_median,ZFeedback1_max,ZFeedback1_mean,ZFeedback1_std,ZFeedback2_min,ZFeedback2_median,ZFeedback2_max,ZFeedback2_mean,ZFeedback2_std,ZFeedback3_min,ZFeedback3_median,ZFeedback3_max,ZFeedback3_mean,ZFeedback3_std,Res1_min,Res1_median,Res1_max,Res1_mean,Res1_std,Res2_min,Res2_median,Res2_max,Res2_mean,...,VoltageLineToLine23_max,VoltageLineToLine23_mean,VoltageLineToLine23_std,VoltageLineToLine31_min,VoltageLineToLine31_median,VoltageLineToLine31_max,VoltageLineToLine31_mean,VoltageLineToLine31_std,SF1_min,SF1_median,SF1_max,SF1_mean,SF1_std,SF2_min,SF2_median,SF2_max,SF2_mean,SF2_std,SF3_min,SF3_median,SF3_max,SF3_mean,SF3_std,VoltSF1_min,VoltSF1_median,VoltSF1_max,VoltSF1_mean,VoltSF1_std,VoltSF2_min,VoltSF2_median,VoltSF2_max,VoltSF2_mean,VoltSF2_std,VoltSF3_min,VoltSF3_median,VoltSF3_max,VoltSF3_mean,VoltSF3_std,ExtSmartFactor1_min,ExtSmartFactor1_median,ExtSmartFactor1_max,ExtSmartFactor1_mean,ExtSmartFactor1_std,ExtSmartFactor2_min,ExtSmartFactor2_median,ExtSmartFactor2_max,ExtSmartFactor2_mean,ExtSmartFactor2_std,ExtSmartFactor3_min,ExtSmartFactor3_median,ExtSmartFactor3_max,ExtSmartFactor3_mean,ExtSmartFactor3_std,Vol_spec_values,current_spec_min,current_spec_max,장입전력원단위,MES_ton,chr_tot_MWH,Year
0,1,E235373,1,7,7_V1,1.0,0.02,43.94,61.60,42.90,15.839241,16.34,47.38,69.94,47.47,14.280442,16.56,44.06,88.30,47.70,15.468072,250.5,412.0,685.5,427.0,92.007348,196.9,391.5,510.8,370.2,91.784763,182.8,400.5,519.5,386.5,96.134902,4.59,9.57,100.00,15.890,23.947211,2.93,8.500,20.70,9.090,4.808204,3.12,9.220,29.70,9.580,5.636401,2.619141,7.740234,100.000000,14.089495,24.381291,1.419922,6.208984,16.812500,6.815534,...,745.5,637.0,65.499089,562.0,655.0,806.0,658.5,54.915748,85,210.0,999,256.392857,219.835181,69,198.0,999,248.714286,192.902366,64,186.5,999,232.964286,175.346743,13.4,43.30,77.7,41.90,12.995019,25.5,39.56,54.3,39.00,7.908605,19.7,35.00,61.0,36.30,8.389638,73.0,490.0,32768.0,2850.46440,8457.939108,123.0,451.0,7540.0,699.25000,1369.273195,113.0,447.5,32768.0,1712.07140,6100.864134,730,53.0,53.0,417.853891,47.4,19.806274,2023
1,1,E235373,1,8,8_V1,2.0,31.05,37.84,44.62,37.84,9.601184,26.38,34.88,43.38,34.88,12.020815,37.03,45.75,54.44,45.75,12.308077,468.5,524.0,579.0,524.0,78.135299,401.5,469.5,537.5,469.5,96.166522,169.9,315.5,461.2,315.5,206.033238,12.97,14.03,15.09,14.030,1.502602,9.26,14.810,20.38,14.810,7.861039,3.12,7.790,12.46,7.790,6.605648,10.210938,11.355469,12.500000,11.355469,1.618612,7.910156,10.615234,13.320312,10.615234,...,738.5,700.0,54.093669,681.0,727.0,772.5,727.0,64.700270,181,225.0,269,225.000000,62.225397,165,306.0,447,306.000000,199.404112,246,247.0,248,247.000000,1.414214,33.4,47.40,61.4,47.40,19.798990,30.7,39.60,48.5,39.60,12.584291,47.4,49.20,51.0,49.20,2.541165,427.0,572.0,717.0,572.00000,205.060967,159.0,633.5,1108.0,633.50000,671.044335,563.0,799.0,1035.0,799.00000,333.754401,760,50.5,50.5,417.853891,47.4,19.806274,2023
2,1,E235373,1,9,9_V1,3.0,13.93,38.97,64.00,38.97,35.405058,14.28,35.97,57.66,35.97,30.670757,10.51,27.70,44.90,27.70,24.323368,329.0,450.8,572.5,450.8,172.180501,404.0,475.8,547.5,475.8,101.469823,396.0,451.5,507.2,451.5,78.665629,5.14,23.12,41.10,23.120,25.422698,9.50,18.900,28.30,18.900,13.291398,8.82,28.530,48.25,28.530,27.880999,2.810547,14.999023,27.187500,14.999023,17.237109,7.679688,16.550781,25.421875,16.550781,...,819.5,768.0,73.185552,655.5,728.5,801.5,728.5,103.237590,285,593.0,901,593.000000,435.577777,251,314.5,378,314.500000,89.802561,238,618.5,999,618.500000,538.108260,27.9

In [23]:
agg_tap_pattern_cnt = agg_arc_alldata[["Year", "Month", "HEAT_NO", "ActualCHRG_PX3", "cnt_change_tap", "patt_change_tap"]].drop_duplicates().groupby(["Year", "Month", "ActualCHRG_PX3", "cnt_change_tap", "patt_change_tap"])["HEAT_NO"].agg(["count"]).reset_index()
agg_tap_pattern_cnt.sort_values(["Year", "ActualCHRG_PX3", "count"], ascending = [True, True, False], inplace = True)
agg_tap_pattern_cnt

,Year,Month,ActualCHRG_PX3,cnt_change_tap,patt_change_tap,count
8,2023,1,1,8,7_8_9_10_11_12_11_10,415
107,2023,3,1,12,7_8_9_10_11_12_11_10_9_8_7_6,376
48,2023,2,1,8,7_8_9_10_11_12_11_10,239
314,2023,9,1,6,7_8_9_10_11_12,206
163,2023,4,1,12,7_8_9_10_11_12_11_10_9_8_7_6,191
...,...,...,...,...,...,...
421,2023,11,3,10,10_9_8_9_10_11_10_9_8_6,1
449,2023,12,3,7,8_11_10_9_8_7_6,1
451,2023,12,3,8,10_9_8_9_10_11_10_6,1
455,2023,12,3,11,10_9_8_9_10_11_10_9_8_7_6,1


In [24]:
agg_tap_pattern_cnt[agg_tap_pattern_cnt["Month"] == 12]

,Year,Month,ActualCHRG_PX3,cnt_change_tap,patt_change_tap,count
439,2023,12,1,12,7_8_9_10_11_12_11_10_9_8_7_6,149
426,2023,12,1,6,7_8_9_10_11_12,87
429,2023,12,1,8,7_8_9_10_11_12_11_10,21
431,2023,12,1,9,7_8_9_10_11_12_11_10_9,17
437,2023,12,1,11,7_8_9_10_11_12_11_10_9_8_7,17
434,2023,12,1,10,7_8_9_10_11_12_11_10_9_8,16
428,2023,12,1,7,7_8_9_10_11_12_11,10
432,2023,12,1,9,7_8_9_10_12_11_10_9_8,2
438,2023,12,1,11,7_8_9_10_12_11_10_9_8_7_6,2
424,2023,12,1,4,7_10_11_12,1


In [25]:
agg_tap_pattern_mwh = agg_arc_alldata[["Year", "Month", "HEAT_NO", "ActualCHRG_PX3", "cnt_change_tap", "patt_change_tap", "charge_전력사용량"]].drop_duplicates().groupby(["Year", "Month", "ActualCHRG_PX3", "cnt_change_tap", "patt_change_tap"]).agg(mwh_median = ("charge_전력사용량", lambda x : np.median(x))).reset_index()
agg_tap_pattern_mwh.sort_values(["Year", "ActualCHRG_PX3", "mwh_median"], ascending = [True, True, False], inplace = True)
agg_tap_pattern_mwh

,Year,Month,ActualCHRG_PX3,cnt_change_tap,patt_change_tap,mwh_median
62,2023,2,1,15,6_7_6_7_8_9_10_11_12_11_10_9_8_7_6,34.812500
14,2023,1,1,12,7_8_9_10_11_12_11_10_9_8_7_6,17.048462
42,2023,2,1,7,7_10_11_12_11_10_9,16.900024
12,2023,1,1,11,7_8_9_10_11_12_11_10_9_8_7,16.853149
9,2023,1,1,9,7_8_9_10_11_12_11_10_8,16.806274
...,...,...,...,...,...,...
454,2023,12,3,10,9_8_9_10_11_10_9_8_7_6,9.802368
278,2023,7,3,11,10_9_8_9_10_11_10_9_8_7_6,9.696899
138,2023,3,3,10,12_11_10_9_8_9_10_11_10_6,8.400024
213,2023,5,3,12,11_10_9_8_9_10_11_10_9_8_7_6,6.599243


In [26]:
agg_tap_pattern_ontime = agg_arc_alldata[["Year", "Month", "HEAT_NO", "ActualCHRG_PX3", "cnt_change_tap", "patt_change_tap", "On_time"]].drop_duplicates().groupby(["Year", "Month", "ActualCHRG_PX3", "cnt_change_tap", "patt_change_tap"]).agg(ontime_median = ("On_time", lambda x : np.median(x))).reset_index()
agg_tap_pattern_ontime.sort_values(["Year", "ActualCHRG_PX3", "ontime_median"], ascending = [True, True, False], inplace = True)
agg_tap_pattern_ontime

,Year,Month,ActualCHRG_PX3,cnt_change_tap,patt_change_tap,ontime_median
90,2023,3,1,2,7_11,543.0
424,2023,12,1,4,7_10_11_12,399.5
57,2023,2,1,11,12_11_10_9_8_7_8_9_10_11_12,167.5
42,2023,2,1,7,7_10_11_12_11_10_9,124.0
50,2023,2,1,9,7_8_9_10_11_12_11_10_12,96.0
...,...,...,...,...,...,...
138,2023,3,3,10,12_11_10_9_8_9_10_11_10_6,38.0
141,2023,3,3,11,10_9_8_9_10_11_10_9_8_7_6,32.0
210,2023,5,3,10,6_8_9_10_11_10_9_8_7_6,30.0
213,2023,5,3,12,11_10_9_8_9_10_11_10_9_8_7_6,27.0


In [27]:
agg_tap_pattern_merge = reduce(lambda x, y : pd.merge(x, y, on = ["Year", "Month", "ActualCHRG_PX3", 
                                                                   "cnt_change_tap", "patt_change_tap"]), 
                               [agg_tap_pattern_cnt, agg_tap_pattern_mwh, agg_tap_pattern_ontime])

# 
agg_tap_pattern_merge.sort_values(["Year", "Month", "ActualCHRG_PX3", "count"], ascending = [True, True, True, False], inplace = True)
agg_tap_pattern_merge

,Year,Month,ActualCHRG_PX3,cnt_change_tap,patt_change_tap,count,mwh_median,ontime_median
0,2023,1,1,8,7_8_9_10_11_12_11_10,415,15.696899,46.5
45,2023,1,1,6,7_8_9_10_11_12,17,14.798462,27.0
72,2023,1,1,12,7_8_9_10_11_12_11_10_9_8_7_6,8,17.048462,28.5
78,2023,1,1,6,7_10_11_12_11_10,6,15.603149,66.0
79,2023,1,1,11,7_8_9_10_11_12_11_10_9_8_7,6,16.853149,75.5
...,...,...,...,...,...,...,...,...
403,2023,12,3,10,9_8_9_10_11_10_9_8_7_6,2,9.802368,69.0
453,2023,12,3,7,8_11_10_9_8_7_6,1,11.798462,84.0
454,2023,12,3,8,10_9_8_9_10_11_10_6,1,12.696899,72.0
455,2023,12,3,11,10_9_8_9_10_11_10_9_8_7_6,1,15.603149,49.5


In [54]:
# os.chdir(r"C:\Users\Administrator\Desktop\Begas\OUT\csv")
# agg_tap_pattern_merge.to_csv(r"2022_2023 월별 전압인가패턴 별 전력_작업시간_중앙값.csv", index = False, encoding = "cp949")

In [7]:
# 전력량 통계량 집계 : 평균적인 차이가 크지 않음  
# mwmeans = agg_tap_pattern_merge.groupby(["Year", "Month", "ActualCHRG_PX3"])["mwh_median"].mean().reset_index()
# mwmeans.pivot_table(index = ["Year", "Month"], columns = "ActualCHRG_PX3", values = "mwh_median")

In [2]:
# charge별 Tap 별 전력량 합계와 charge별 총 전력량 비교 
# agg_arc_alldata["sums_MWH_value"] = agg_arc_alldata.groupby(["HEAT_NO", "ActualCHRG_PX3"])["MWH_value"].transform(lambda x : sum(x))
# agg_arc_alldata["mwh_check_diff"] = agg_arc_alldata["sums_MWH_value"] - agg_arc_alldata["charge_전력사용량"]

In [1]:
# 59 HEAT가 일치하지 않음
# len(pd.unique(agg_arc_alldata[agg_arc_alldata["mwh_check_diff"] != 0]["HEAT_NO"]))

In [3]:
# agg_arc_alldata[agg_arc_alldata["mwh_check_diff"] != 0]

In [4]:
# 크게 차이나는 경우 다수 : 대부분 SMART ARC 전력량 데이터 첫 수집 시 이상 데이터 수집 때문으로 보여짐
# agg_arc_alldata[agg_arc_alldata["mwh_check_diff"] != 0]["mwh_check_diff"].unique()

In [5]:
# agg_arc_alldata[agg_arc_alldata["mwh_check_diff"] < -18]

In [6]:
# agg_arc_alldata[agg_arc_alldata["HEAT_NO"] == "E240014"][["HEAT_NO", "ActualCHRG_PX3", "전압번호순서_Tag", "charge_전력사용량", "MWH_value", "sums_MWH_value"]]
# agg_arc_alldata[agg_arc_alldata["HEAT_NO"] == "E235954"][["HEAT_NO", "ActualCHRG_PX3", "전압번호순서_Tag", "charge_전력사용량", "MWH_value", "sums_MWH_value"]]

In [139]:
# pp = agg_arc_2022[(agg_arc_2022["ActualCHRG_PX3"] == 1)]
# pp = pp[pp["Month"] == 8]

In [138]:
# pp = agg_arc_2022[(agg_arc_2022["ActualCHRG_PX3"] == 1)]
# chk = pp[pp["HEAT_NO"] == "E230825"][["ActFceTap_Px3", "전압번호순서_Tag", "On_time", "charge_전력사용량", "MWH_value"]]
# display(chk)
# print(chk["MWH_value"].sum())

In [93]:
# pp["HEAT_NO"].unique()

### Columns

In [28]:
# 항목 선택하기 
PK_cols = ["Year", "Month", "HEAT_NO", "ActualCHRG_PX3", 
           "ActFceTap_Px3", "전압번호순서_Tag", "전압인가순서", "patt_change_tap", "check_order", "UD_tag", "Tap_tag"] 
aggr_cols = [x for x in agg_arc_2023.columns if "median" in x] + ["MWH_value", "On_time", "burner_value", "lance_value", "chrgc_value", "injtc_value"]
added_cols = ["Vol_spec_values", "current_spec_min", "current_spec_max", "장입전력원단위"]
selected_cols_reaggregate = PK_cols + aggr_cols + added_cols

### Tap 패턴 반영한 집계 : 2022, 2023 all

In [29]:
chr_pat_data_full = []
for chr_n in [1, 2, 3]:
    
    # 
    print("Full data Charge : ", chr_n)
    chr_n_data = agg_arc_alldata[agg_arc_alldata["ActualCHRG_PX3"] == chr_n]
    
    # 
    aggr_ls = []
    for colname in aggr_cols :
        
        # 
        subset_dt = chr_n_data[PK_cols + added_cols + [colname]]
        reshape_data = subset_dt.melt(id_vars = PK_cols + added_cols)
        reshape_data["charge"] = ["Charge" + str(x) for x in reshape_data["ActualCHRG_PX3"]]
        reshape_data["column_var"] = ["_".join([x, y, z,]) for x, y, z, in zip(reshape_data["charge"],
                                                                                reshape_data["Tap_tag"],
                                                                                reshape_data["UD_tag"])]
        
        # reordering
        reshape_data.sort_values(by = ["Year", "Month", "HEAT_NO", "ActualCHRG_PX3", "전압인가순서"], 
                                 ascending = [True, True, True, True, True], inplace = True)
        reshape_data["agg_value_mean"] = reshape_data.groupby(["Year", "Month", "ActualCHRG_PX3", 
                                                               "patt_change_tap", "Tap_tag", "UD_tag"])["value"].transform(lambda x : np.mean(x))
        
        # 
        wide_data = reshape_data.pivot_table(index = ["Year", "Month", "ActualCHRG_PX3", "variable", "patt_change_tap"], 
                                             columns = ["column_var"], 
                                             values = ["agg_value_mean"]).reset_index()
        wide_data.columns = ["_".join(pair) for pair in wide_data.columns]
        wide_data.rename(columns = {"Year_" : "Year", 
                                    "Month_" : "Month", 
                                    "ActualCHRG_PX3_" : "ActualCHRG_PX3", 
                                    "variable_" : "variable",
                                    "patt_change_tap_" : "patt_change_tap"}, inplace = True)
        wide_data.columns = [re.sub("agg_value_mean_", "", x) for x in wide_data.columns]
        
        
        #display(wide_data)

        # 
        aggr_ls.append(wide_data)

    ;
    
    agg_chr_n_data = pd.concat(aggr_ls, axis = 0)
    
    # 
    chr_pat_data_full.append(agg_chr_n_data)
    
;


Full data Charge :  1
Full data Charge :  2
Full data Charge :  3


''

###  Charge별 데이터 병합


In [30]:
# charge data 
aggregate_patt_added_charge_1 = chr_pat_data_full[0]
aggregate_patt_added_charge_2 = chr_pat_data_full[1]
aggregate_patt_added_charge_3 = chr_pat_data_full[2]

# 결측치 보정 
aggregate_patt_added_charge_1 = aggregate_patt_added_charge_1.fillna(0)
aggregate_patt_added_charge_2 = aggregate_patt_added_charge_2.fillna(0)
aggregate_patt_added_charge_3 = aggregate_patt_added_charge_3.fillna(0)


# 데이터 병합
aggregate_patt_added_charge_1 = pd.merge(agg_tap_pattern_merge[["Year", "Month", "ActualCHRG_PX3", 
                                                                "patt_change_tap", "cnt_change_tap", "count"]].drop_duplicates(), 
                                         aggregate_patt_added_charge_1, 
                                         on = ["Year", "Month", "ActualCHRG_PX3", "patt_change_tap"])


aggregate_patt_added_charge_2 = pd.merge(agg_tap_pattern_merge[["Year", "Month", "ActualCHRG_PX3", 
                                                                "patt_change_tap", "cnt_change_tap", "count"]].drop_duplicates(), 
                                         aggregate_patt_added_charge_2, 
                                         on = ["Year", "Month", "ActualCHRG_PX3", "patt_change_tap"])


aggregate_patt_added_charge_3 = pd.merge(agg_tap_pattern_merge[["Year", "Month", "ActualCHRG_PX3", 
                                                                "patt_change_tap", "cnt_change_tap", "count"]].drop_duplicates(), 
                                         aggregate_patt_added_charge_3, 
                                         on = ["Year", "Month", "ActualCHRG_PX3", "patt_change_tap"])


In [31]:
aggregate_patt_added_charge_3

,Year,Month,ActualCHRG_PX3,patt_change_tap,cnt_change_tap,count,variable,Charge3_Tap10_Down,Charge3_Tap10_Up,Charge3_Tap11_Down,Charge3_Tap11_Up,Charge3_Tap12_Up,Charge3_Tap6_Down,Charge3_Tap6_Up,Charge3_Tap7_Down,Charge3_Tap7_Up,Charge3_Tap8_Down,Charge3_Tap8_Up,Charge3_Tap9_Down,Charge3_Tap9_Up
0,2023,1,3,8_9_10_11_10_9_8_7_6,9,419,Current1_median,44.223484,31.086659,0.0,41.073771,0.0,49.057749,0.0,42.881277,0.0,44.462470,42.873879,44.550561,37.482978
1,2023,1,3,8_9_10_11_10_9_8_7_6,9,419,Current2_median,46.455442,32.434286,0.0,43.160239,0.0,51.270585,0.0,44.785024,0.0,46.439009,44.306062,46.722064,38.909625
2,2023,1,3,8_9_10_11_10_9_8_7_6,9,419,Current3_median,45.971981,31.392320,0.0,42.859379,0.0,50.915506,0.0,44.015370,0.0,45.741683,43.881217,45.926146,38.328258
3,2023,1,3,8_9_10_11_10_9_8_7_6,9,419,Voltage1_median,460.174940,485.279117,0.0,491.638663,0.0,369.805012,0.0,390.093675,0.0,415.238425,437.125060,434.625060,455.327446
4,2023,1,3,8_9_10_11_10_9_8_7_6,9,419,Voltage2_median,427.281862,473.244630,0.0,456.860143,0.0,339.688305,0.0,366.506324,0.0,388.696181,419.798568,407.255609,444.900239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2023,12,3,11_10_9_8_9_10_11_10_9_8_7_6,12,1,On_time,170.000000,3.000000,0.0,91.500000,0.0,653.000000,0.0,3.000000,0.0,15.000000,0.000000,3.000000,0.000000
9995,2023,12,3,11_10_9_8_9_10_11_10_9_8_7_6,12,1,burner_value,12.500000,0.000000,0.0,29.000000,0.0,44.000000,0.0,1.000000,0.0,1.500000,0.000000,0.500000,0.000000
9996,2023,12,3,11_10_9_8_9_10_11_10_9_8_7_6,12,1,lance_value,161.000000,5.000000,0.0,43.500000,0.0,348.000000,0.0,7.000000,0.0,11.500000,0.000000,3.000000,2.000000
9997,2023,12,3,11_10_9_8_9_10_11_10_9_8_7_6,12,1,chrgc_value,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [27]:
On_time = aggregate_patt_added_charge_1[aggregate_patt_added_charge_1["variable"] == "On_time"]
On_time[On_time["count"] > 50]

,Year,Month,ActualCHRG_PX3,patt_change_tap,cnt_change_tap,count,variable,Charge1_Tap10_Down,Charge1_Tap10_Up,Charge1_Tap11_Down,Charge1_Tap11_Up,Charge1_Tap12_Down,Charge1_Tap12_Up,Charge1_Tap6_Down,Charge1_Tap6_Up,Charge1_Tap7_Down,Charge1_Tap7_Up,Charge1_Tap8_Down,Charge1_Tap8_Up,Charge1_Tap9_Down,Charge1_Tap9_Up
56,2022,4,1,7_8_9_10_11_12_11_10,8,521,On_time,94.449136,25.522073,4.990403,4.393474,0.0,851.357006,0.000000,0.0,0.000000,83.318618,0.000000,4.262956,0.000000,3.988484
1093,2022,5,1,7_8_9_10_11_12_11_10,8,546,On_time,101.489011,26.635531,4.989011,4.357143,0.0,873.347985,0.000000,0.0,0.000000,89.706960,0.000000,4.280220,0.000000,3.998168
1947,2022,6,1,7_8_9_10_11_12_11_10,8,219,On_time,106.027397,25.369863,4.977169,4.328767,0.0,862.155251,0.000000,0.0,0.000000,87.940639,0.000000,4.269406,0.000000,4.000000
2618,2022,7,1,7_8_9_10_11_12_11_10,8,410,On_time,98.614634,26.224390,4.982927,4.256098,0.0,846.397561,0.000000,0.0,0.000000,84.517073,0.000000,4.202439,0.000000,3.990244
3472,2022,8,1,7_8_9_10_11_12_11_10,8,275,On_time,97.196364,26.785455,4.992727,4.214545,0.0,857.938182,0.000000,0.0,0.000000,86.880000,0.000000,4.174545,0.000000,3.996364
4143,2022,9,1,7_8_9_10_11_12_11_10,8,351,On_time,74.991453,25.082621,4.980057,4.216524,0.0,871.430199,0.000000,0.0,0.000000,87.225071,0.000000,4.185185,0.000000,4.000000
4997,2022,10,1,7_8_9_10_11_12_11_10,8,331,On_time,60.592145,25.885196,4.996979,4.274924,0.0,897.906344,0.000000,0.0,0.000000,84.610272,0.000000,4.193353,0.000000,3.990937
5058,2022,10,1,7_8_9_10_11_12,6,182,On_time,0.000000,27.703297,0.000000,4.225275,0.0,883.494505,0.000000,0.0,0.000000,85.241758,0.000000,4.153846,0.000000,3.978022
5851,2022,11,1,7_8_9_10_11_12_11_10,8,286,On_time,59.853147,25.199301,4.968531,4.304196,0.0,925.804196,0.000000,0.0,0.000000,86.006993,0.000000,4.206294,0.000000,4.000000
6705,2022,12,1,7_8_9_10_11_12_11_10,8,174,On_time,63.959770,24.637931,2.367816,3.471264,0.0,940.540230,0.000000,0.0,0.000000,82.936782,0.000000,3.390805,0.000000,3.316092


In [32]:
# os.chdir(r"C:\Users\Administrator\Desktop\Begas\OUT\csv")

aggregate_patt_added_charge_1.to_csv(r"2022_2023 월별 Smart Arc 항목 전압구간별 집계_전압변동패턴반영_Charge 1.csv", index = False, encoding = "cp949")
aggregate_patt_added_charge_2.to_csv(r"2022_2023 월별 Smart Arc 항목 전압구간별 집계_전압변동패턴반영_Charge 2.csv", index = False, encoding = "cp949")
aggregate_patt_added_charge_3.to_csv(r"2022_2023 월별 Smart Arc 항목 전압구간별 집계_전압변동패턴반영_Charge 3.csv", index = False, encoding = "cp949")